In [92]:
# paramter cell do not remove!!
nb_parm='llmnok'
embed_model = "mxbai-embed-large" 
gen_model = "deepseek-r1:7b"
collection = "Bridknowledge"
question = "give me a species_code that have scientific_name equal to Cairina moschata and leatest taxonomic_sort_order"

In [93]:
import sys
import os

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

## Do the task After this

In [94]:
print("nb_parm: ",nb_parm)
print("embed_model: ",embed_model)
print("gen_model: ",gen_model)
print("collection: ",collection)
print("question: ",question)

nb_parm:  llmnok
embed_model:  mxbai-embed-large
gen_model:  deepseek-r1:7b
collection:  Bridknowledge
question:  give me a species_code that have scientific_name equal to Cairina moschata and leatest taxonomic_sort_order


## Download module

In [95]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
import requests
import json
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Provide highest score in simmilarity search

In [96]:
def get_highest_score_sim(docs):
    # Tokenize each document
    tokenized_documents = [word_tokenize(page.lower()) for page in docs]
    
    # Initialize the BM25 model
    bm25 = BM25Okapi(tokenized_documents)
    
    # Tokenize the query
    tokenized_query = word_tokenize(question.lower())
    
    # Get the BM25 scores for each document based on the query
    scores = bm25.get_scores(tokenized_query)
    
    # Rank the documents by score
    return sorted(zip(scores, docs), reverse=True)

## Vector db retrieval

In [97]:
import weaviate
import json
from datetime import datetime
from weaviate.classes.query import MetadataQuery, Filter
from weaviate.classes.query import Filter

# # search1
def search_weaviate(Query):
    client = Utility.registerClient()    
    retriever = client.collections.get(collection)
    

    response = retriever.query.bm25( # search without model
        # query="Conclusion",
        query = Query,
        limit=1,
        query_properties=["content"],
        return_metadata=MetadataQuery(score=True),
    )
    client.close()  # Free up resources
    for obj in response.objects:
        result =  {
            "Title": {obj.properties['title']},
            "Content": obj.properties['content']
        }

        return result


# # search 2
# def search_weaviate(Query):
#     client = Utility.registerClient()    
#     retriever = client.collections.get(collection)

#     # find the right doc title
#     response_doc_title = retriever.query.fetch_objects(
#         filters=Filter.by_property("source_type").equal("summary"),
#         limit=500
#     )
#     docs = []
#     for obj in response_doc_title.objects:
#         content = obj.properties['content'].replace('\n\n', '')
#         docs.append(f"Title: {obj.properties['title']}, Summary: {content}")
    
#     ranked_docs = get_highest_score_sim(docs)
#     doc_title = ranked_docs[0][1].split(",")[0].split("Title: ")[-1]
    
#     # search on specify doc_title
#     response = retriever.query.bm25( # search without model
#         query = Query,
#         limit=3,
#         query_properties=["content"],
#         filters=Filter.by_property("title").equal(doc_title),
#         return_metadata=MetadataQuery(score=True),
#     )
#     client.close()  # Free up resources
#     result = {}
#     for obj in response.objects:
#         if "Title" in result:
#             result["Content"] += "\n\n" + obj.properties['content']
#         else:
#             result["Title"] = obj.properties['title']
#             result["Content"] = obj.properties['content']
        
#     return result


print(search_weaviate(question))


{'Title': {'PFW_spp_translation_table_May2024.csv'}, 'Content': 'musduc | nan | 10 | Cairina moschata | Muscovy Duck | 2023 | 423'}


## Wikipedia free api

In [123]:
def search_wikipedia(query):
    # Step 1: Search for the most relevant page
    search_url = "https://en.wikipedia.org/w/api.php"
    search_params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json",
        "srlimit": 50 
    }
    search_response = requests.get(search_url, params=search_params)
    search_data = search_response.json()

    if not search_data["query"]["search"]:
        return "No search results found."

    # Step 2: Get the title of the highest match with questions
    # use snippet to filter web page
    ###############################################################
    list_of_snippets = [f"Title: {i['title']}, Snippet: {i['snippet']}" for i in search_data["query"]["search"]] # nested list
    ranked_docs = get_highest_score_sim(list_of_snippets)


    page_title = ranked_docs[0][1].split(",")[0].split("Title: ")[-1]
    # Step 3: Fetch the extract/content of that page
    content_params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,
        "titles": page_title,
        "format": "json"
    }
    content_response = requests.get(search_url, params=content_params)
    content_data = content_response.json()

    pages = content_data["query"]["pages"]
    for page_id, page in pages.items():
        result =  {
            "Title": {page_title},
            "Content": page.get("extract", "No content found.")
        }

        return result

# Example usage
# print(search_wikipedia(question))


No search results found.


## Check Relevant Documents

In [99]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def format_docs(docs): # result in long text type str
    # return "\n\n".join(doc.page_content for doc in docs)
    return RunnableLambda(lambda _: "\n\n".join(doc for doc in docs))



# Question
# result = rag_chain.invoke(question)
# print(f"\nFianl Answer: {result}")
# print(result.split("Answer:")[-1].strip().split()[0])


In [100]:
from typing import TypedDict, List

class GraphState(TypedDict):
    question: str
    documents: List[str]
    grade: str
    generation: str
 

In [101]:
from langchain.schema import Document
import re

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = search_wikipedia(question)
    # web_results = Document(page_content=docs['Content'])

    return {"documents": docs['Content'], "question": question}
    
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = search_weaviate(question)
    return {"documents": documents['Content'], "question": question}

def grade_documents(state):
    """
    Determines whether the retrieved document are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates document key with only relevant document
    """
    
    question = state["question"]
    documents = state["documents"]

    prompt = PromptTemplate.from_template(
        """
        You are a grader assessing relevance of a retrieved document to a user question.

        If the document contains keyword(s) or semantic meaning related to the question, grade it as **relevant**.

        ONLY return a single word: **yes** or **no** — do not explain your reasoning.

        Context:
        {context}

        Question:
        {question}

        Answer:
        """
    )
    
    # LLM
    llm = OllamaLLM(
        model="deepseek-r1:7b",
        temperature=0,
        base_url="http://host.docker.internal:11434" 
    )
    
    # Chain
    rag_chain = (
        {"context": format_docs([documents]), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    try:
        result = rag_chain.invoke(question)
        print(f"old format : {result.split('Answer:')[-1].strip().split()[0].lower()}")
    
        # Extract yes or no
        match = re.search(r'\b(yes|no)\b', result.strip().lower())
        grade = match.group(1) if match else "no"
    except:
        match = re.search(r'\b(yes|no)\b', result.strip().lower())
        grade = match.group(1) if match else "no"


    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print(f"Grade result: {grade}")
    print("###################################################################")
    print(result)
    print("###################################################################")

    return {"documents": documents, "question": question, "grade": grade }


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    
    print("---DECIDE TO GENERATE---")
    question = state["question"]
    filtered_documents = state["documents"]
    grade = state["grade"].lower()
    print(f"Grade result: {grade}")

    if grade == 'no':
        print("---DECISION: DOCUMENT ARE NOT RELEVANT TO QUESTION, WEB SEARCH---")
        return "web_search"
    else:
        print("---DECISION: DOCUMENT ARE RELEVANT GO CHECK HALLUCINATION---")
        return "check_hallucinate"

def check_hallucinate(state):
    question = state["question"]
    documents = state["documents"]
    
    prompt = PromptTemplate.from_template(
       """
       You are a grader assessing whether an LLM generation is grounded in and supported by a set of retrieved facts.

       ONLY respond with a single word: **yes** or **no**.
       - "yes" = the generation is grounded in the context.
       - "no" = the generation is not grounded or is hallucinated.

       Context:
       {context}

       Question:
       {question}

       Answer:
       """
    )
    
    # LLM
    llm = OllamaLLM(
        model="deepseek-r1:7b",
        temperature=0,
        base_url="http://host.docker.internal:11434" 
    )
    
    # Chain
    rag_chain = (
        {"context": format_docs([documents]), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    result = rag_chain.invoke(question)

    try:
        result = rag_chain.invoke(question)
        print(f"old format : {result.split('Answer:')[-1].strip().split()[0].lower()}")
    
        # Extract yes or no
        match = re.search(r'\b(yes|no)\b', result.strip().lower())
        grade = match.group(1) if match else "no"
    except:
        match = re.search(r'\b(yes|no)\b', result.strip().lower())
        grade = match.group(1) if match else "no"
        
    print("---CHECKING HALLUCINATION---")
    print(f"Grade result: {grade}")
    print("###################################################################")
    print(result)
    print("###################################################################")

    return {"documents": documents, "question": question, "grade": grade }
        
def decide_final_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """
    
    print("---DECIDE FINAL GENERATION---")
    question = state["question"]
    filtered_documents = state["documents"]
    grade = state["grade"].lower()
    print(f"Grade result: {grade}")
    
    if grade == "yes":
        print("---DECISION: GENERATION ADDRESSES QUESTION---")
        return "useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not useful"
        
def final_generate(state):
    prompt = PromptTemplate.from_template(
        """
        You are a helpful AI assistant. Based on the provided context, generate a concise and accurate answer to the question.
        
        Context:
        {context}
    
        Question:
        {question}
    
        Answer (based only on the context above):
        """
    )
    
    # LLM
    llm = OllamaLLM(
        model="deepseek-r1:7b",
        temperature=0,
        base_url="http://host.docker.internal:11434" 
    )
    
    question = state["question"]
    documents = state["documents"]
    # Chain
    rag_chain = (
        {"context": format_docs([documents]), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    print("---FINAL GENERATE---")

    # RAG generation
    generation = rag_chain.invoke(question)
    print(f"FINAL ANSWER: {generation}")
    return {"documents": documents, "question": question, "generation": generation}
    

        
def llm_fallback(state):
    """
    Generate answer using the LLM w/o vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---LLM Fallback---")
    question = state["question"]
    
    prompt = PromptTemplate.from_template(
       """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. Use three sentences maximum and keep the answer concise."""
    )
    
    # LLM
    llm = OllamaLLM(
        model="deepseek-r1:7b",
        temperature=0,
        base_url="http://host.docker.internal:11434" 
    )
    
    # Chain
    rag_chain = (
        {"question": RunnablePassthrough()}  # Add this step
        | prompt
        | llm
        | StrOutputParser()
    )

    generation = rag_chain.invoke({"question": question})
    
    return {"question": question, "generation": generation}


In [102]:
import pprint

from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("check_hallucinate", check_hallucinate)  # check_hallucinate
workflow.add_node("web_search", web_search)  # web_search
workflow.add_node("llm_fallback", llm_fallback)  # llm_fallback
workflow.add_node("final_generate", final_generate)  # final_generate


# Add edges
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("web_search", "grade_documents")

# Add conditional edges from grade_documents
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "check_hallucinate": "check_hallucinate",
    },
)

# Add conditional edges from check_hallucinate
workflow.add_conditional_edges(
    "check_hallucinate",
    decide_final_generate,
    {
        "not useful": "web_search",
        "useful": "final_generate",
    },
)
workflow.add_edge("final_generate", END)

# Optionally handle fallback path
workflow.add_edge("llm_fallback", END)
    

# Compile
app = workflow.compile()

In [103]:
# Run
inputs = {"question": question}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(str(value["generation"]))

---RETRIEVE---
"Node 'retrieve':"
'\n---\n'
old format : <think>
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Grade result: yes
###################################################################
<think>
Okay, so I need to determine if the document is relevant to the user's question. The context given includes several pieces of information: musduc, nan, 10, Cairina moschata, Muscovy Duck, 2023, and 423. 

The user's question is asking for a species_code where the scientific_name is equal to Cairina moschata, specifically looking for the latest taxonomic_sort_order. So they want the most recent one.

Looking at the context, I see that Cairina moschata is mentioned as Muscovy Duck. The numbers 2023 and 423 are present. It's possible that these numbers represent years or identifiers related to species data. Since the user is asking for the latest taxonomic_sort_order, which likely refers to the most recent version or update number, the presence of 423 might be the identifier they're looking

In [122]:
for valus in output.values():
    print(v['documents'])
    print('\n\n')
    print(v['question'])
    print('\n\n')
    print(v['generation'])
    print('\n\n')


musduc | nan | 10 | Cairina moschata | Muscovy Duck | 2023 | 423



give me a species_code that have scientific_name equal to Cairina moschata and leatest taxonomic_sort_order



<think>
Okay, so I need to figure out the species code for a specific scientific name. The question is asking for a species_code where the scientific_name is "Cairina moschata" and it should be the latest taxonomic_sort_order as of 2023.

Looking at the context provided: musduc | nan | 10 | Cairina moschata | Muscovy Duck | 2023 | 423

I see that "Cairina moschata" is paired with the number 423. The other fields are musduc, nan, and 10, but I'm not sure what those represent. Since the question is about species_code, which likely refers to a code assigned by the International Union for Conservation of Nature (IUCN), I think that's where the 423 comes from.

The taxonomic_sort_order probably refers to the order in which species are classified. The latest one would be the most recent or highest rank. Since 423 is